In [1]:
import numpy as np
import torch
import pandas as pd
import networkx as nx
import hypernetx as hnx
import itertools
from sklearn.cluster import KMeans
import tensorly as tl

In [2]:
# Generate a random tensor
n_nodes = 100
m = 3
K = 3
dim = [K] * m

# Randomly assign each node to a cluster
clusters = np.random.randint(0,K,n_nodes)

# One-hot encoding of the clusters
one_hot = np.zeros((n_nodes,K))
for i in range(n_nodes):
    one_hot[i,clusters[i]] = 1
one_hot = torch.tensor(one_hot)

P_hat = torch.zeros(dim)
for i in range(K):
    for j in range(i,K):
        for k in range(j,K):
            # Random variable uniformly distributed between 0 and 1
            p = np.random.uniform()
            P_hat[i,j,k] = p
            P_hat[i,k,j] = p
            P_hat[j,i,k] = p
            P_hat[j,k,i] = p
            P_hat[k,i,j] = p
            P_hat[k,j,i] = p

In [3]:
# Print the size of P_hat and one_hot
print(P_hat.size())
print(one_hot.shape)

torch.Size([3, 3, 3])
torch.Size([100, 3])


In [4]:
one_hot

tensor([[0., 0., 1.],
        [0., 0., 1.],
        [0., 0., 1.],
        [1., 0., 0.],
        [0., 0., 1.],
        [0., 1., 0.],
        [1., 0., 0.],
        [1., 0., 0.],
        [0., 0., 1.],
        [1., 0., 0.],
        [1., 0., 0.],
        [0., 0., 1.],
        [0., 0., 1.],
        [0., 1., 0.],
        [0., 1., 0.],
        [0., 0., 1.],
        [0., 1., 0.],
        [1., 0., 0.],
        [0., 0., 1.],
        [1., 0., 0.],
        [0., 1., 0.],
        [0., 0., 1.],
        [0., 1., 0.],
        [0., 0., 1.],
        [0., 0., 1.],
        [0., 0., 1.],
        [1., 0., 0.],
        [0., 1., 0.],
        [1., 0., 0.],
        [0., 1., 0.],
        [0., 1., 0.],
        [0., 0., 1.],
        [0., 0., 1.],
        [0., 1., 0.],
        [0., 0., 1.],
        [1., 0., 0.],
        [0., 1., 0.],
        [0., 1., 0.],
        [0., 1., 0.],
        [0., 1., 0.],
        [0., 0., 1.],
        [1., 0., 0.],
        [0., 0., 1.],
        [1., 0., 0.],
        [0., 1., 0.],
        [0

In [4]:
# normalize each column of one_hot
one_hot = one_hot.float()
one_hot = one_hot / (one_hot.sum(0) ** 0.5)
one_hot

tensor([[0.0000, 0.0000, 0.1690],
        [0.0000, 0.1741, 0.0000],
        [0.0000, 0.0000, 0.1690],
        [0.0000, 0.0000, 0.1690],
        [0.0000, 0.1741, 0.0000],
        [0.0000, 0.0000, 0.1690],
        [0.1768, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.1690],
        [0.1768, 0.0000, 0.0000],
        [0.1768, 0.0000, 0.0000],
        [0.1768, 0.0000, 0.0000],
        [0.0000, 0.1741, 0.0000],
        [0.1768, 0.0000, 0.0000],
        [0.1768, 0.0000, 0.0000],
        [0.1768, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.1690],
        [0.0000, 0.0000, 0.1690],
        [0.0000, 0.1741, 0.0000],
        [0.1768, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.1690],
        [0.1768, 0.0000, 0.0000],
        [0.0000, 0.1741, 0.0000],
        [0.0000, 0.1741, 0.0000],
        [0.0000, 0.0000, 0.1690],
        [0.1768, 0.0000, 0.0000],
        [0.1768, 0.0000, 0.0000],
        [0.0000, 0.1741, 0.0000],
        [0.0000, 0.0000, 0.1690],
        [0.0000, 0.0000, 0.1690],
        [0.176

In [37]:
# Check if one_hot has orthogonal columns
one_hot.T @ one_hot

tensor([[31.,  0.,  0.],
        [ 0., 37.,  0.],
        [ 0.,  0., 32.]], dtype=torch.float64)

In [5]:
tl.set_backend('pytorch')
P_hat = P_hat.float()
one_hot = one_hot.float()
Q = tl.tucker_to_tensor((P_hat, [one_hot,one_hot,one_hot]))

In [6]:
Q.shape

torch.Size([100, 100, 100])

In [7]:
import tensorflow as tf
from scipy.stats import ortho_group

def n_mode_product(x, u, n):
    n = int(n)
    # We need one letter per dimension
    # (maybe you could find a workaround for this limitation)
    if n > 26:
        raise ValueError('n is too large.')
    ind = ''.join(chr(ord('a') + i) for i in range(n))
    exp = f'{ind}K...,JK->{ind}J...'
    result = tf.einsum(exp, x, u)
    return torch.tensor(result.numpy())

In [8]:
def multi_mode_dot(tensor, matrices, modes):
    res = tensor
    for mode, matrix in zip(modes, matrices):
        res = n_mode_product(res, matrix, mode)
    return res

In [9]:
def unfold(tensor, mode):
    
    return torch.reshape(torch.moveaxis(tensor, mode, 0), (tensor.shape[mode], -1))

In [10]:
def HOOI(tensor, ranks, n_iter_max=1000, tol=1e-8):
    """High-Order Orthogonal Iteration (HOOI) algorithm for Tucker decomposition
    Perform Higher Order Orthogonal Iteration (HOOI) on a 3D tensor.

    Parameters:
    - tensor: Input tensor of size I x J x K.
    - ranks: Tuple of target ranks (r1, r2, r3).
    - n_iter_max: Maximum number of iterations.
    - eps: Convergence criterion.

    Returns:
    - core_tensor: Core tensor of the Tucker decomposition.
    - [L, R, V] factor matrices.
    L ∈ R^I×r1, R ∈ R^J×r2, V ∈ R^K×r3.
    """
    # Initialize R and V factor matrices with orthonormal columns
    I, J, K = tensor.shape
    R = ortho_group.rvs(J)[:,:ranks[1]]
    R_hat = torch.tensor(R, dtype=torch.float32)
    V = ortho_group.rvs(K)[:,:ranks[2]]
    V_hat = torch.tensor(V, dtype=torch.float32)

    for _ in range(n_iter_max):
        # C = A ×2 R^T ×3 V^T
        c = multi_mode_dot(tensor, [R_hat.T, V_hat.T], modes=[1, 2])
        c_1 = unfold(c, 0)

        # L = SVD(r1, C_1), where U = SVD(k, C) means compute the k’th order truncated 
        # SVD of C and then set U = [u1, u2, . . . , uk] to the matrix whose columns are 
        # the k largest left singular vectors ui of C
        u, _, _ = torch.svd(c_1)
        L_hat = u[:, :ranks[0]]

        # D = A ×1 L^T ×3 V^T
        d = multi_mode_dot(tensor, [L_hat.T, V_hat.T], modes=[0, 2])
        d_2 = unfold(d, 1)

        #R = SVD(r2, D_2)
        u, _, _ = torch.svd(d_2)
        R_hat = u[:, :ranks[1]]

        # E = A ×1 L^T ×2 R^T
        e = multi_mode_dot(tensor, [L_hat.T, R_hat.T], modes=[0, 1])
        e_3 = unfold(e, 2)

        # V = SVD(r3, E_3)
        u, _, _ = torch.svd(e_3)
        V_hat = u[:, :ranks[2]]

        # Compute the approximation error
        core_tensor = n_mode_product(e, V_hat.T, 2)
        appr = multi_mode_dot(core_tensor, [L_hat, R_hat, V_hat], modes=[0, 1, 2])
        if torch.norm(tensor - appr) < tol:
            break

    return core_tensor, [L_hat, R_hat, V_hat]


In [11]:
# Apply the Tucker decomposition to find the factor matrices
core, factors = HOOI(Q, ranks=[K,K,K], n_iter_max=10000, tol=1e-8)

In [12]:
def scale_invariant(matrix):
    """Scale-invariant tensor, proposed the following row-wise normalization on given matrix
    Parameters
    ----------
    matrix : torch tensor
    Returns
    -------
    torch tensor
        scale-invariant matrix
    """
    # For each row, divide by the first coordinate of the row
    return matrix / matrix[:, 0].unsqueeze(1)

In [13]:
# The adjacency tensor must be symmetric, then the factor matrices are same, so we can use the first one
factor = factors[0]

# Apply scale-invariant to the factor matrix
factor = scale_invariant(factor)
R_hat = factor[:, 1:]

# Apply K-mean Clustering to the rows of factor matrix
kmeans = KMeans(n_clusters=K, random_state=0).fit(R_hat)
labels = kmeans.labels_

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/cluster/_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


In [14]:
labels

array([2, 2, 2, 1, 2, 0, 1, 1, 2, 1, 1, 2, 2, 0, 0, 2, 0, 1, 2, 1, 0, 2,
       0, 2, 2, 2, 1, 0, 1, 0, 0, 2, 2, 0, 2, 1, 0, 0, 0, 0, 2, 1, 2, 1,
       0, 2, 0, 0, 2, 2, 1, 2, 1, 0, 0, 0, 1, 2, 2, 1, 2, 0, 2, 0, 1, 1,
       2, 2, 0, 1, 2, 1, 2, 2, 2, 0, 0, 1, 0, 1, 2, 0, 0, 1, 2, 2, 1, 2,
       2, 0, 1, 0, 1, 2, 1, 0, 0, 1, 0, 0], dtype=int32)

In [18]:
# Take all the indices whose label is 0
indices_0 = [i for i, x in enumerate(labels) if x == 0]

# Take all the indices whose label is 1
indices_1 = [i for i, x in enumerate(labels) if x == 1]

# Take all the indices whose label is 2
indices_2 = [i for i, x in enumerate(labels) if x == 2]


In [15]:
clusters

array([2, 2, 2, 0, 2, 1, 0, 0, 2, 0, 0, 2, 2, 1, 1, 2, 1, 0, 2, 0, 1, 2,
       1, 2, 2, 2, 0, 1, 0, 1, 1, 2, 2, 1, 2, 0, 1, 1, 1, 1, 2, 0, 2, 0,
       1, 2, 1, 1, 2, 2, 0, 2, 0, 1, 1, 1, 0, 2, 2, 0, 2, 1, 2, 1, 0, 0,
       2, 2, 1, 0, 2, 0, 2, 2, 2, 1, 1, 0, 1, 0, 2, 1, 1, 0, 2, 2, 0, 2,
       2, 1, 0, 1, 0, 2, 0, 1, 1, 0, 1, 1])

In [16]:
# Take all the indices whose cluster is 2
clusters_0 = [i for i, x in enumerate(clusters) if x == 1]

# Take all the indices whose cluster is 0
clusters_1 = [i for i, x in enumerate(clusters) if x == 0]

# Take all the indices whose cluster is 1
clusters_2 = [i for i, x in enumerate(clusters) if x == 2]

In [19]:
# Check how many same indices are in the indices_0 and clusters_0
print(len(set(indices_0) & set(clusters_0)) / len(set(indices_0) | set(clusters_0)))

# Check how many same indices are in the indices_1 and clusters_1
print(len(set(indices_1) & set(clusters_1)) / len(set(indices_1) | set(clusters_1)))

# Check how many same indices are in the indices_2 and clusters_2
print(len(set(indices_2) & set(clusters_2)) / len(set(indices_2) | set(clusters_2)))

1.0
1.0
1.0


In [51]:
accuracy = np.sum(labels == clusters) / n_nodes
print(accuracy)

1.0


In [20]:
A = multi_mode_dot(core, factors, modes=[0, 1, 2])
torch.norm(Q - A)

tensor(0.0005)

In [21]:
torch.norm(factors[0] - factors[1])

tensor(4.2383e-07)

In [22]:
torch.norm(factors[0] - factors[2])

tensor(3.8658e-07)

In [23]:
torch.norm(factors[0] - one_hot)

tensor(10.5468)

In [26]:
one_hot

tensor([[0., 0., 1.],
        [0., 0., 1.],
        [0., 0., 1.],
        [1., 0., 0.],
        [0., 0., 1.],
        [0., 1., 0.],
        [1., 0., 0.],
        [1., 0., 0.],
        [0., 0., 1.],
        [1., 0., 0.],
        [1., 0., 0.],
        [0., 0., 1.],
        [0., 0., 1.],
        [0., 1., 0.],
        [0., 1., 0.],
        [0., 0., 1.],
        [0., 1., 0.],
        [1., 0., 0.],
        [0., 0., 1.],
        [1., 0., 0.],
        [0., 1., 0.],
        [0., 0., 1.],
        [0., 1., 0.],
        [0., 0., 1.],
        [0., 0., 1.],
        [0., 0., 1.],
        [1., 0., 0.],
        [0., 1., 0.],
        [1., 0., 0.],
        [0., 1., 0.],
        [0., 1., 0.],
        [0., 0., 1.],
        [0., 0., 1.],
        [0., 1., 0.],
        [0., 0., 1.],
        [1., 0., 0.],
        [0., 1., 0.],
        [0., 1., 0.],
        [0., 1., 0.],
        [0., 1., 0.],
        [0., 0., 1.],
        [1., 0., 0.],
        [0., 0., 1.],
        [1., 0., 0.],
        [0., 1., 0.],
        [0

In [27]:
factor

tensor([[ 1.0000, -2.3583, -0.1519],
        [ 1.0000, -2.3583, -0.1519],
        [ 1.0000, -2.3583, -0.1519],
        [ 1.0000,  0.5180, -1.4584],
        [ 1.0000, -2.3583, -0.1519],
        [ 1.0000,  0.3714,  0.8176],
        [ 1.0000,  0.5180, -1.4584],
        [ 1.0000,  0.5180, -1.4584],
        [ 1.0000, -2.3583, -0.1519],
        [ 1.0000,  0.5180, -1.4584],
        [ 1.0000,  0.5180, -1.4584],
        [ 1.0000, -2.3583, -0.1519],
        [ 1.0000, -2.3583, -0.1519],
        [ 1.0000,  0.3714,  0.8176],
        [ 1.0000,  0.3714,  0.8176],
        [ 1.0000, -2.3583, -0.1519],
        [ 1.0000,  0.3714,  0.8176],
        [ 1.0000,  0.5180, -1.4584],
        [ 1.0000, -2.3583, -0.1519],
        [ 1.0000,  0.5180, -1.4584],
        [ 1.0000,  0.3714,  0.8176],
        [ 1.0000, -2.3583, -0.1519],
        [ 1.0000,  0.3714,  0.8176],
        [ 1.0000, -2.3583, -0.1519],
        [ 1.0000, -2.3583, -0.1519],
        [ 1.0000, -2.3583, -0.1519],
        [ 1.0000,  0.5180, -1.4584],
 

In [24]:
torch.norm(core - P_hat)

tensor(533.0045)